In [163]:
import pandas as pd

#loading data
df = pd.read_csv('task-hours-december24-data.csv', sep = ";", encoding='UTF-8')

#displaying the first 5 rows
print(df.head())

#displaying the last 5 rows
print(df.tail())

#displaying the shape of the dataframe
print(df.shape)

#displaying the columns of the dataframe
print(df.columns)



     Category     User  Start Date Start Time    End Date  End Time  \
0     PRZERWA  Klaudia  11.12.2024   12:46:52  11.12.2024  13:10:40   
1   OBDZWONKA  Klaudia  11.12.2024   13:12:37  11.12.2024  13:15:46   
2  OBSLUGA PH  Klaudia  11.12.2024   13:17:14  11.12.2024  13:23:59   
3     PRZERWA  Klaudia  11.12.2024   13:26:45  11.12.2024  13:37:39   
4   OBDZWONKA  Natalia  11.12.2024   13:28:51  11.12.2024  13:30:01   

  Duration (h) Duration (decimal)    DAY  
0     00:23:48                0,4  SRODA  
1     00:03:09               0,05  SRODA  
2     00:06:45               0,11  SRODA  
3     00:10:54               0,18  SRODA  
4     00:01:10               0,02  SRODA  
              Category     User  Start Date Start Time    End Date  End Time  \
397             WIZYTA  Natalia  19.12.2024   11:13:10  19.12.2024  11:49:47   
398               MAIL  Natalia  19.12.2024   11:49:48  19.12.2024  12:15:50   
399             WIZYTA  Natalia  19.12.2024   12:15:51  19.12.2024  13:06:2

In [164]:
# Convert date columns from DD.MM.YYYY format
df['Start Date'] = pd.to_datetime(df['Start Date'], format='%d.%m.%Y')
df['End Date'] = pd.to_datetime(df['End Date'], format='%d.%m.%Y')

# Convert time columns from HH:mm:ss format
df['Start Time'] = pd.to_datetime(df['Start Time'], format='%H:%M:%S').dt.time
df['End Time'] = pd.to_datetime(df['End Time'], format='%H:%M:%S').dt.time

# Convert Duration (h) from HH:mm:ss to timedelta
df['Duration (h)'] = pd.to_timedelta(df['Duration (h)'])

# Display sample data to verify conversions
print("Sample of converted data:")
print(df[['Start Date', 'Start Time', 'End Date', 'End Time', 'Duration (h)']].head())

# Display data types to verify conversions
print("\nData types after conversion:")
print(df[['Start Date', 'Start Time', 'End Date', 'End Time', 'Duration (h)']].dtypes)

Sample of converted data:
  Start Date Start Time   End Date  End Time    Duration (h)
0 2024-12-11   12:46:52 2024-12-11  13:10:40 0 days 00:23:48
1 2024-12-11   13:12:37 2024-12-11  13:15:46 0 days 00:03:09
2 2024-12-11   13:17:14 2024-12-11  13:23:59 0 days 00:06:45
3 2024-12-11   13:26:45 2024-12-11  13:37:39 0 days 00:10:54
4 2024-12-11   13:28:51 2024-12-11  13:30:01 0 days 00:01:10

Data types after conversion:
Start Date       datetime64[ns]
Start Time               object
End Date         datetime64[ns]
End Time                 object
Duration (h)    timedelta64[ns]
dtype: object


In [165]:
# Number of unique users
unique_users_count = df['User'].nunique()
print(f"Number of unique users: {unique_users_count}")

Number of unique users: 3


In [166]:
# Grouping dates by weekday
df['DZIEN_TYGODNIA'] = df['Start Date'].dt.day_name()

# Couting unique dates for Fridays and other days
fridays = df[df['DAY'] == 'PIATEK']['Start Date'].nunique()
weekdays = df[df['DAY'] != 'PIATEK']['Start Date'].nunique()

# Couting working hours

friday_hours = fridays * 5
weekday_hours = weekdays * 8
working_hours = friday_hours + weekday_hours

print(f"Fridays: {fridays}")
print(f"Weekdays: {weekdays}")
print(f"Working hours: {working_hours}")

Fridays: 1
Weekdays: 6
Working hours: 53


In [167]:
# Fix column name from DZIEN_TYGODNIA to DAY
df['DAY'] = df['Start Date'].dt.day_name()

# Working time analysis
time_per_person = df.groupby(['User', 'Category'])['Duration (h)'].sum().reset_index()
time_per_person['Duration (h)'] = time_per_person['Duration (h)'].apply(lambda x: str(x))

# Calculate total reported time per person
total_reported = df.groupby('User')['Duration (h)'].sum()

# Calculate difference between expected and reported time
working_hours_timedelta = pd.Timedelta(hours=working_hours)
not_recorded = pd.Series(working_hours_timedelta - total_reported, name='not_recorded')

# Calculate daily averages
working_days = fridays + weekdays
daily_average_total = working_hours_timedelta / working_days
daily_reported_per_person = total_reported / working_days
daily_not_recorded = not_recorded / working_days

# Display working days information
print("Working days details:")
print(f"Number of Fridays (fridays): {fridays}")
print(f"Number of other working days (weekdays): {weekdays}")
print(f"Total working days (working_days): {working_days}")

print("\nDaily averages:")
print(f"Expected daily average (daily_average_total): {daily_average_total}")
print("\nDaily reported average per person (daily_reported_per_person):")
print(daily_reported_per_person)
print("\nDaily not recorded average (daily_not_recorded):")
print(daily_not_recorded)

# Additional helper information
print("\nValue of working_hours_timedelta:")
print(working_hours_timedelta)
print("\nValue of total_reported:")
print(total_reported)

Working days details:
Number of Fridays (fridays): 1
Number of other working days (weekdays): 6
Total working days (working_days): 7

Daily averages:
Expected daily average (daily_average_total): 0 days 07:34:17.142857142

Daily reported average per person (daily_reported_per_person):
User
Klaudia    0 days 06:57:33.285714285
Natalia              0 days 05:44:27
Patrycja   0 days 05:57:29.571428571
Name: Duration (h), dtype: timedelta64[ns]

Daily not recorded average (daily_not_recorded):
User
Klaudia    0 days 00:36:43.857142857
Natalia    0 days 01:49:50.142857142
Patrycja   0 days 01:36:47.571428571
Name: not_recorded, dtype: timedelta64[ns]

Value of working_hours_timedelta:
2 days 05:00:00

Value of total_reported:
User
Klaudia    2 days 00:42:53
Natalia    1 days 16:11:09
Patrycja   1 days 17:42:27
Name: Duration (h), dtype: timedelta64[ns]


In [168]:
# Calculate time spent on each category by user
time_by_category = df.groupby(['User', 'Category'])['Duration (h)'].agg(['sum', 'count']).reset_index()

# Rename columns for clarity
time_by_category.columns = ['User', 'Category', 'Total Hours', 'Count']

# Convert timedelta to hours as float for better readability
time_by_category['Total Hours'] = time_by_category['Total Hours'].dt.total_seconds() / 3600

# Sort values by User and Total Hours
time_by_category = time_by_category.sort_values(['User', 'Total Hours'], ascending=[True, False])

# Calculate percentage of time spent on each category per user
total_time_per_user = time_by_category.groupby('User')['Total Hours'].sum()
time_by_category['Percentage'] = time_by_category.apply(
    lambda x: (x['Total Hours'] / total_time_per_user[x['User']]) * 100, 
    axis=1
)

# Format the output
time_by_category['Total Hours'] = time_by_category['Total Hours'].round(2)
time_by_category['Percentage'] = time_by_category['Percentage'].round(2)

# Display results
print("Time analysis by user and category:")
print("\nDetailed breakdown:")
print(time_by_category.to_string(index=False))

# Display summary per user
print("\nTotal hours per user:")
print(total_time_per_user.round(2))

Time analysis by user and category:

Detailed breakdown:
    User                       Category  Total Hours  Count  Percentage
 Klaudia                      OBDZWONKA        12.56     50       25.77
 Klaudia                           MAIL        10.48     74       21.51
 Klaudia                  ODDELEGOWANIE        10.20      3       20.95
 Klaudia                     OBSLUGA PH         6.31     44       12.96
 Klaudia                        PRZERWA         4.07     29        8.35
 Klaudia                         OFERTY         1.92     10        3.93
 Klaudia                          LEADY         1.77      7        3.63
 Klaudia                      PRZETARGI         1.33      8        2.73
 Klaudia                         WIZYTA         0.08      1        0.17
 Natalia                      OBDZWONKA        16.82     20       41.85
 Natalia                         WIZYTA        11.68     10       29.06
 Natalia                           MAIL         3.44      6        8.55
 Natali

In [169]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define a color mapping for each category with more muted colors
category_colors = { 
    'PRZERWA': '#A9A9A9',  # Dark Gray
    'OBDZWONKA': '#4682B4',  # Steel Blue
    'OBSLUGA PH': '#8A2BE2',  # Blue Violet
    'MAIL': '#FF6347',  # Tomato
    'OFERTY': '#8B4513',  # Saddle Brown
    'WIZYTA': '#3CB371',  # Medium Sea Green
    'LEADY': '#FFB6C1',  # Light Pink
    'TEST': '#D3D3D3',  # Light Gray
    'B/D': '#FFA500',  # Orange
    'AUDYT': '#00CED1',  # Dark Turquoise
    'PRZYGOTOWANIE STANOWISKA PRACY': '#FF69B4',  # Hot Pink
    'PRZETARGI': '#FF4500',  # Orange Red
    'DOKUMENTY': '#90EE90',  # Light Green
    'PORZADKI W BIURZE': '#FFFACD',  # Lemon Chiffon
    'ODDELEGOWANIE': '#9370DB',  # Medium Purple
}

# Get unique users
users = time_by_category['User'].unique()

# First create summary plots for all users combined
all_users_data = time_by_category.groupby('Category').agg({
    'Total Hours': 'sum',
    'Count': 'sum'
}).reset_index()

# Create summary figure with two subplots
summary_fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "bar"}, {"type": "pie"}]],
    subplot_titles=(
        'Total Time Spent on Each Category (All Users)',
        'Percentage Distribution of Tasks (All Users)'
    )
)

# Add summary bar chart
summary_fig.add_trace(
    go.Bar(
        x=all_users_data['Category'],
        y=all_users_data['Total Hours'],
        text=all_users_data['Total Hours'].round(2),
        textposition='auto',
        name='Hours',
        marker_color=[category_colors[cat] for cat in all_users_data['Category']]  # Set colors
    ),
    row=1, col=1
)

# Add summary pie chart
summary_fig.add_trace(
    go.Pie(
        values=all_users_data['Total Hours'],
        labels=all_users_data['Category'],
        textinfo='percent+label',
        hovertemplate="Category: %{label}<br>" +
                     "Hours: %{value:.2f}<br>" +
                     "Percentage: %{percent}<br>" +
                     "<extra></extra>",
        marker=dict(colors=[category_colors[cat] for cat in all_users_data['Category']])  # Set colors
    ),
    row=1, col=2
)

# Update summary layout
total_hours_all = all_users_data['Total Hours'].sum()
summary_fig.update_layout(
    title={
        'text': f'Time Analysis for All Users<br>' +
               f'<sup>Total Hours: {total_hours_all:.2f}</sup>',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    height=600,
    width=1200,
    showlegend=False,
    bargap=0.2
)

# Update summary axes
summary_fig.update_xaxes(
    title_text='Category',
    tickangle=45,
    row=1, col=1
)
summary_fig.update_yaxes(
    title_text='Hours',
    row=1, col=1
)

# Show summary plots
summary_fig.show()

# Create individual plots for each user
for user in users:
    user_data = time_by_category[time_by_category['User'] == user]
    
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{"type": "bar"}, {"type": "pie"}]],
        subplot_titles=(
            f'Time Spent on Each Category - {user}',
            f'Percentage Distribution of Tasks - {user}'
        )
    )
    
    # Add bar chart
    fig.add_trace(
        go.Bar(
            x=user_data['Category'],
            y=user_data['Total Hours'],
            text=user_data['Total Hours'].round(2),
            textposition='auto',
            name='Hours',
            marker_color=[category_colors[cat] for cat in user_data['Category']]  # Set colors
        ),
        row=1, col=1
    )
    
    # Add pie chart
    fig.add_trace(
        go.Pie(
            values=user_data['Total Hours'],
            labels=user_data['Category'],
            textinfo='percent+label',
            hovertemplate="Category: %{label}<br>" +
                         "Hours: %{value:.2f}<br>" +
                         "Percentage: %{percent}<br>" +
                         "<extra></extra>",
            marker=dict(colors=[category_colors[cat] for cat in user_data['Category']])  # Set colors
        ),
        row=1, col=2
    )
    
    # Update layout
    total_hours = user_data['Total Hours'].sum()
    fig.update_layout(
        title={
            'text': f'Time Analysis for {user}<br>' +
                   f'<sup>Total Hours: {total_hours:.2f}</sup>',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        height=600,
        width=1200,
        showlegend=False,
        bargap=0.2
    )
    
    # Update axes
    fig.update_xaxes(
        title_text='Category',
        tickangle=45,
        row=1, col=1
    )
    fig.update_yaxes(
        title_text='Hours',
        row=1, col=1
    )
    
    fig.show()

# Create stacked bar chart comparing all users
comparison_fig = go.Figure()

for category in time_by_category['Category'].unique():
    category_data = time_by_category[time_by_category['Category'] == category]
    comparison_fig.add_trace(
        go.Bar(
            name=category,
            x=category_data['User'],
            y=category_data['Total Hours'],
            text=category_data['Total Hours'].round(2),
            textposition='auto',
            hovertemplate="User: %{x}<br>" +
                         "Category: " + category + "<br>" +
                         "Hours: %{y:.2f}<br>" +
                         "<extra></extra>",
            marker_color=category_colors[category]  # Set color
        )
    )

# Update layout for comparison chart
comparison_fig.update_layout(
    title={
        'text': 'Comparison of Time Distribution Across Users',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='User',
    yaxis_title='Hours',
    barmode='stack',
    height=700,
    width=1200,
    bargap=0.2,
    legend_title='Category',
    legend={
        'orientation': 'h',
        'yanchor': 'bottom',
        'y': -0.2,
        'xanchor': 'center',
        'x': 0.5
    }
)

# Create summary distribution chart for all users combined
summary_distribution_fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "bar"}, {"type": "pie"}]],
    subplot_titles=(
        'Distribution of Time Across Categories (All Users Combined)',
        'Percentage Distribution (All Users Combined)'
    )
)

# Calculate total hours per category for all users
total_by_category = time_by_category.groupby('Category')['Total Hours'].sum().reset_index()

# Add bar chart
summary_distribution_fig.add_trace(
    go.Bar(
        x=total_by_category['Category'],
        y=total_by_category['Total Hours'],
        text=total_by_category['Total Hours'].round(2),
        textposition='auto',
        name='Hours',
        marker_color=[category_colors[cat] for cat in total_by_category['Category']]  # Set colors
    ),
    row=1, col=1
)

# Add pie chart
summary_distribution_fig.add_trace(
    go.Pie(
        values=total_by_category['Total Hours'],
        labels=total_by_category['Category'],
        textinfo='percent+label',
        hovertemplate="Category: %{label}<br>" +
                     "Hours: %{value:.2f}<br>" +
                     "Percentage: %{percent}<br>" +
                     "<extra></extra>",
        marker=dict(colors=[category_colors[cat] for cat in total_by_category['Category']])  # Set colors
    ),
    row=1, col=2
)

# Update layout
total_hours = total_by_category['Total Hours'].sum()
summary_distribution_fig.update_layout(
    title={
        'text': f'Time Distribution Summary - All Users Combined<br>' +
               f'<sup>Total Hours: {total_hours:.2f}</sup>',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    height=600,
    width=1200,
    showlegend=False,
    bargap=0.2
)

summary_distribution_fig.update_xaxes(
    title_text='Category',
    tickangle=45,
    row=1, col=1
)
summary_distribution_fig.update_yaxes(
    title_text='Hours',
    row=1, col=1
)

summary_distribution_fig.show()

# Create stacked bar chart comparing all users with improved labels
comparison_fig = go.Figure()

# Calculate total hours per user for labels
total_hours_per_user = time_by_category.groupby('User')['Total Hours'].sum()

# Add bars for each category
for category in time_by_category['Category'].unique():
    category_data = time_by_category[time_by_category['Category'] == category]
    comparison_fig.add_trace(
        go.Bar(
            name=category,
            x=category_data['User'],
            y=category_data['Total Hours'],
            text=[f"{category}<br>{hours:.2f}h" for hours in category_data['Total Hours']],  # Combine category and hours
            textposition='inside',  # Position text inside the bars
            hovertemplate="User: %{x}<br>" +
                         "Category: " + category + "<br>" +
                         "Hours: %{y:.2f}<br>" +
                         "<extra></extra>",
            marker_color=category_colors[category]  # Set color
        )
    )

# Update layout for comparison chart
comparison_fig.update_layout(
    title={
        'text': 'Time Distribution Comparison Between Users',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='User',
    yaxis_title='Hours',
    barmode='stack',
    height=700,  # Increase height for better visibility
    width=1200,
    bargap=0.2,
    showlegend=False  # Hide the legend
)

# Update axes with better visibility
comparison_fig.update_xaxes(
    tickangle=45,
    tickfont=dict(size=12)
)

# Show the updated comparison plot
comparison_fig.show()

# Calculate not recorded time for each user
not_recorded_time = working_hours_timedelta - total_reported

# Create a DataFrame for not recorded time
not_recorded_df = not_recorded_time.reset_index()
not_recorded_df.columns = ['User', 'Not Recorded Hours']

# Convert timedelta to hours and round to 1 decimal place
not_recorded_df['Not Recorded Hours'] = (not_recorded_df['Not Recorded Hours'].dt.total_seconds() / 3600).round(1)

# Create a bar chart for not recorded time
not_recorded_fig = go.Figure()

not_recorded_fig.add_trace(
    go.Bar(
        x=not_recorded_df['User'],
        y=not_recorded_df['Not Recorded Hours'],  # Use the rounded values
        text=not_recorded_df['Not Recorded Hours'],  # Display rounded values
        textposition='auto',
        name='Not Recorded Hours'
    )
)

# Update layout for not recorded time chart
not_recorded_fig.update_layout(
    title={
        'text': 'Not Recorded Time for Each User',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='User',
    yaxis_title='Hours',
    height=600,
    width=1200,
    bargap=0.2
)

# Update axes
not_recorded_fig.update_xaxes(tickangle=45)

# Show not recorded time plot
not_recorded_fig.show()

# Combine 'End Date' and 'End Time' into a full timestamp
df['End Time'] = df.apply(lambda row: pd.Timestamp.combine(row['End Date'], row['End Time']), axis=1)

# Extract hours from 'End Time'
df['Hour'] = df['End Time'].dt.hour

# Define time bins (e.g., hourly)
time_bins = list(range(8, 18))  # Bins from 8 to 17
time_labels = [f"{hour}:00" for hour in time_bins[:-1]]  # Labels for bins

# Create plots for each user
users = df['User'].unique()

for user in users:
    user_data = df[df['User'] == user]
    
    # Count occurrences for each category in time bins
    user_data['Time Bin'] = pd.cut(user_data['Hour'], bins=time_bins, labels=time_labels, right=False)
    category_counts = user_data.groupby(['Time Bin', 'Category']).size().unstack(fill_value=0)

    # Create bar chart
    fig = go.Figure()

    for category in category_counts.columns:
        fig.add_trace(
            go.Bar(
                x=category_counts.index,
                y=category_counts[category],
                name=category,
                text=category_counts[category],
                textposition='auto',
                marker_color=category_colors[category]  # Set color
            )
        )

    # Chart settings
    fig.update_layout(
        title=f'Category Occurrences by Time Bin for {user}',
        xaxis_title='Time Bin',
        yaxis_title='Occurrences',
        barmode='stack',
        height=600,
        width=1200,
        legend_title='Category'
    )

    # Show chart
    fig.show()

C:\Users\d.rajczak\AppData\Local\Temp\ipykernel_6256\3709411610.py:400: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\d.rajczak\AppData\Local\Temp\ipykernel_6256\3709411610.py:401: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



C:\Users\d.rajczak\AppData\Local\Temp\ipykernel_6256\3709411610.py:400: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\d.rajczak\AppData\Local\Temp\ipykernel_6256\3709411610.py:401: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



C:\Users\d.rajczak\AppData\Local\Temp\ipykernel_6256\3709411610.py:400: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\d.rajczak\AppData\Local\Temp\ipykernel_6256\3709411610.py:401: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

